In [0]:
%pip install azure-eventhub azure-eventhub-checkpointstoreblob-aio

  Using cached azure_eventhub-5.15.1-py3-none-any.whl.metadata (30 kB)
  Using cached azure_eventhub_checkpointstoreblob_aio-1.2.0-py3-none-any.whl.metadata (13 kB)
  Using cached aiohttp-3.13.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (8.1 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.8.0-cp312-cp312-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl.metadata (20 kB)
  Using cached multidict-6.7.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached propcache-0.4.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (13 kB)
  Using cached yarl-1.22.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (75

In [0]:
# CONFIG: acceso a Storage Account (ADLS Gen2)
spark.conf.set(
    "fs.azure.account.key.stmeteoend2end.dfs.core.windows.net",
    "TU_KEY"
)

# IMPORTS
import asyncio
import json
from azure.eventhub.aio import EventHubConsumerClient
from pyspark.sql import Row

# Rutas y conexión
CONNECTION_STR = "TU_CONNECTION_STRING"
EVENTHUB_NAME = "eh-weather"
BRONZE_PATH = "abfss://datalake@stmeteoend2end.dfs.core.windows.net/bronze/streaming/weather_python"


In [0]:
buffer = []

async def on_event(partition_context, event):
    global buffer
    try:
        data = json.loads(event.body_as_str())
        buffer.append(Row(**data))

        if len(buffer) >= 10:  # Microbatch de 10 eventos
            df = spark.createDataFrame(buffer)
            df.write.format("delta").mode("append").save(BRONZE_PATH)
            buffer = []
        
        await partition_context.update_checkpoint(event)
        print("Evento procesado:", data)

    except Exception as e:
        print("Error procesando evento:", e)


async def main():
    client = EventHubConsumerClient.from_connection_string(
        CONNECTION_STR,
        consumer_group="$Default",
        eventhub_name=EVENTHUB_NAME,
    )

    async with client:
        await client.receive(
            on_event=on_event,
            starting_position="-1"
        )

In [0]:
await main()

Evento procesado: {'timestamp': '2025-12-09T21:38:43.086799+00:00', 'ip': '192.168.1.50', 'temperature': 17.03, 'humidity': 60.89, 'pm25': 16.29, 'light': 600, 'uv_level': 0, 'pressure': 1020.58, 'rain_raw': 457, 'wind_raw': 17.15, 'vibration': 3}
Evento procesado: {'timestamp': '2025-12-09T21:38:53.314333+00:00', 'ip': '192.168.1.50', 'temperature': 29.17, 'humidity': 52.17, 'pm25': 51.44, 'light': 267, 'uv_level': 1, 'pressure': 1018.17, 'rain_raw': 293, 'wind_raw': 2.2, 'vibration': 2}
Evento procesado: {'timestamp': '2025-12-09T21:39:03.440840+00:00', 'ip': '192.168.1.50', 'temperature': 16.08, 'humidity': 42.68, 'pm25': 52.82, 'light': 113, 'uv_level': 6, 'pressure': 990.68, 'rain_raw': 412, 'wind_raw': 17.66, 'vibration': 3}
Evento procesado: {'timestamp': '2025-12-09T21:39:13.565989+00:00', 'ip': '192.168.1.50', 'temperature': 29.45, 'humidity': 41.01, 'pm25': 9.17, 'light': 749, 'uv_level': 1, 'pressure': 1008.77, 'rain_raw': 741, 'wind_raw': 1.36, 'vibration': 0}
Evento proces

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can